# Chapter 5 Cross-Validation Applied Exercises

### 5. In Chapter 4, we used logistic regression to predict the probability of default using income and balance on the Default data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from patsy import dmatrices
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import metrics
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis as LDA, QuadraticDiscriminantAnalysis as QDA)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

#### Import data

In [10]:
# Import the Default data
df_default = pd.read_csv(r'..\Data sets\Default.csv')

# Let's start practising creating copies of the original data:
df_default_raw = df_default

#### Show the dataframe

In [4]:
df_default

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


#### Create dummy variables

In [11]:
df_default = pd.get_dummies(df_default, dtype=np.float64).drop(['default_No', 'student_No'], axis=1)
df_default

,balance,income,default_Yes,student_Yes
0,729.526495,44361.625074,0.0,0.0
1,817.180407,12106.134700,0.0,1.0
2,1073.549164,31767.138947,0.0,0.0
3,529.250605,35704.493935,0.0,0.0
4,785.655883,38463.495879,0.0,0.0
...,...,...,...,...
9995,711.555020,52992.378914,0.0,0.0
9996,757.962918,19660.721768,0.0,0.0
9997,845.411989,58636.156984,0.0,0.0
9998,1569.009053,36669.112365,0.0,0.0


#### (a) Fit a logistic regression model that uses income and balance to predict default.

In [72]:
# Split the model into an X an y set:
X = df_default[['balance','income']]
y = df_default['default_Yes']

# Set up the model
logist_default = LogisticRegression()

# Fit the model
results_logist_default = logist_default.fit(X,y)

# Make predictions
y_pred_default_logist = results_logist_default.predict(X)

#### I need the confusion matrix and metrics functions I wrote for Chapter 4:

In [25]:
def confusion_matrix(confusion_matrix):
    """Return a confusion matrix that is formatted neatly. Takes the sklearn confusion matrix as input and return better formatted confusion matrix.
    Wikepedia's convention is followed: https://en.wikipedia.org/wiki/Confusion_matrix"""
    TP = confusion_matrix[1,1] # True positives
    FP = confusion_matrix[0,1] # False positives
    TN = confusion_matrix[0,0] # True negatives
    FN = confusion_matrix[1,0] # False negatives

    Actual_1_sum = TP + FN # Row 1 sum
    Actual_0_sum = FP + TN # Row 2 sum
    Predict_1_sum = TP + FP # Column 1 sum
    Predict_0_sum = FN + TN # Column 2 sum

    if (Actual_1_sum + Actual_0_sum) != (Predict_1_sum + Predict_0_sum):
        print("There is an error.")
    else:
        Total = Actual_0_sum + Actual_1_sum # Total

    confusion_df = pd.DataFrame({'Predicted 1': [TP,FP,Predict_1_sum],
                                 'Predicted 0': [FN,TN,Predict_0_sum],
                                 'Total': [Actual_1_sum,Actual_0_sum, Total],
                                 '': ['Actual 1', 'Actual 0', 'Total']}).set_index('')
    return confusion_df

def confusion_matrix_metrics(y_true,y_pred,confusion_matrix,decimal_places,col_name):
    """Show all the metrics of the confusion matrix in a neatly formatted way. Uses the sklearn metrics functions.
    Inputs are y_actual, y_pred, confusion_matrix from sklearn, decimal places, and name of the model"""

    # Get metrics
    accuracy = round(metrics.accuracy_score(y_true,y_pred),decimal_places)
    precision = round(metrics.precision_score(y_true,y_pred),decimal_places)
    recall = round(metrics.recall_score(y_true,y_pred),decimal_places)
    F_score = round(metrics.f1_score(y_true,y_pred),decimal_places)
    auc = round(metrics.roc_auc_score(y_true,y_pred),decimal_places)

    # Define true positives, false positives, true negatives, and false negatives
    TP = confusion_matrix[1,1] # True positives
    FP = confusion_matrix[0,1] # False positives
    TN = confusion_matrix[0,0] # True negatives
    FN = confusion_matrix[1,0] # False negatives

    # Sum these values
    Actual_1_sum = TP + FN # Row 1 sum
    Actual_0_sum = FP + TN # Row 2 sum
    Predict_1_sum = TP + FP # Column 1 sum
    Predict_0_sum = FN + TN # Column 2 sum
    
    # Calculate some more metrics
    true_positive_rate = round(TP/(TP+FN),decimal_places)
    false_positive_rate = round(FP/(FP+TN),decimal_places)

    # Create row names
    rows = ['Accuracy', 'Precision', 'Recall', 'F-score', 'True positive rate', 'False positive rate','Area under ROC curve']

    # Create column values
    columns = [accuracy,precision,recall,F_score,true_positive_rate,false_positive_rate,auc]     

    # Create a dataframe to display the values
    result = pd.DataFrame({str(col_name):columns},index=rows)
    return result

#### Confusion matrix

In [27]:
cm_default_logist = metrics.confusion_matrix(y, y_pred_default_logist)
confusion_matrix(cm_default_logist)

,Predicted 1,Predicted 0,Total
,,,
Actual 1,108,225,333
Actual 0,38,9629,9667
Total,146,9854,10000


#### Confusion matrix metrics

In [28]:
confusion_matrix_metrics(y, y_pred_default_logist, cm_default_logist, 4, 'Logistic Regression')

,Logistic Regression
Accuracy,0.9737
Precision,0.7397
Recall,0.3243
F-score,0.4509
True positive rate,0.3243
False positive rate,0.0039
Area under ROC curve,0.6602


#### (b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:
i. Split the sample set into a training set and a validation set.

ii. Fit a multiple logistic regression model using only the training observations.

iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.

iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

#### (c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

In [61]:
for i in range(1,4,1):
    # Split into a train and test set:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)

    # Set up the model
    logist_default = LogisticRegression()

    # Fit the model
    results_logist_default = logist_default.fit(X_train,y_train)

    # Make predictions
    y_pred_default_logist = results_logist_default.predict(X_test)
    
    print('Random state ' + str(i))
    cm = metrics.confusion_matrix(y_test, y_pred_default_logist)
    print('')
    display(confusion_matrix(cm))
    print('')
    display(confusion_matrix_metrics(y_test, y_pred_default_logist, cm, 4, 'Logistic Regression'))
    print('')
    print('For random state ' + str(i) + ' the misclassified predictions are ' + str(cm[0,1]) + ' + ' + str(cm[1,0]) + ' = ' + str(cm[0,1]+cm[1,0]) + '. The fraction of misclassified observations are ' + str(round((cm[0,1]+cm[1,0])/cm.sum(),4)) + '.')

Random state 1



,Predicted 1,Predicted 0,Total
,,,
Actual 1,33,58,91
Actual 0,16,2893,2909
Total,49,2951,3000


,Logistic Regression
Accuracy,0.9753
Precision,0.6735
Recall,0.3626
F-score,0.4714
True positive rate,0.3626
False positive rate,0.0055
Area under ROC curve,0.6786



For random state 1 the misclassified predictions are 16 + 58 = 74. The fraction of misclassified observations are 0.0247.
Random state 2



,Predicted 1,Predicted 0,Total
,,,
Actual 1,23,53,76
Actual 0,18,2906,2924
Total,41,2959,3000


,Logistic Regression
Accuracy,0.9763
Precision,0.5610
Recall,0.3026
F-score,0.3932
True positive rate,0.3026
False positive rate,0.0062
Area under ROC curve,0.6482



For random state 2 the misclassified predictions are 18 + 53 = 71. The fraction of misclassified observations are 0.0237.
Random state 3



,Predicted 1,Predicted 0,Total
,,,
Actual 1,37,69,106
Actual 0,6,2888,2894
Total,43,2957,3000


,Logistic Regression
Accuracy,0.9750
Precision,0.8605
Recall,0.3491
F-score,0.4966
True positive rate,0.3491
False positive rate,0.0021
Area under ROC curve,0.6735



For random state 3 the misclassified predictions are 6 + 69 = 75. The fraction of misclassified observations are 0.025.


Different splits of the training data yield slightly different results. The fraction of misclassified results are still quite close, near 0.02.

#### (d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [64]:
# Split the model into an X an y set:
X = df_default[['balance','income','student_Yes']]
y = df_default['default_Yes']

# Split the model into a train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Set up the model
logist_default_5d = LogisticRegression()

# Fit the model
results_logist_default_5d = logist_default_5d.fit(X_train,y_train)

# Make predictions
y_pred_default_logist_5d = results_logist_default_5d.predict(X_test)


#### Confusion matrix

In [66]:
cm_default_logist_5b = metrics.confusion_matrix(y_test, y_pred_default_logist_5d)
confusion_matrix(cm_default_logist_5b)

,Predicted 1,Predicted 0,Total
,,,
Actual 1,19,75,94
Actual 0,20,2886,2906
Total,39,2961,3000


#### Confusion matrix metrics

In [67]:
confusion_matrix_metrics(y_test, y_pred_default_logist_5d, cm_default_logist_5b, 4, 'Logistic Regression')

,Logistic Regression
Accuracy,0.9683
Precision,0.4872
Recall,0.2021
F-score,0.2857
True positive rate,0.2021
False positive rate,0.0069
Area under ROC curve,0.5976


In [68]:
print('The misclassified predictions are ' + str(cm_default_logist_5b[0,1]) + ' + ' + str(cm_default_logist_5b[1,0]) + ' = ' + str(cm_default_logist_5b[0,1]+cm_default_logist_5b[1,0]) + '. The fraction of misclassified observations are ' + str(round((cm_default_logist_5b[0,1]+cm_default_logist_5b[1,0])/cm_default_logist_5b.sum(),4)) + '.')

The misclassified predictions are 20 + 75 = 95. The fraction of misclassified observations are 0.0317.


It doesn't look like adding student to the model improved the results in such a way to get too excited. 

### 6. We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the *Default* data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the glm() function. Do not forget to set a random seed before beginning your analysis.

#### (a) Using the summary() and glm() functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors.

In [77]:
# Fit a logistic regression to predict default by using income and balance
f_6a = 'default_Yes ~ income + balance'

# Set up the model
model_logist_6a = sm.Logit.from_formula(formula=f_6a, data=df_default)
results_logist_6a = model_logist_6a.fit()

display(results_logist_6a.summary())

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            default_Yes   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Sat, 30 Mar 2024   Pseudo R-squ.:                  0.4594
Time:                        08:25:31   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.5405      0.435    -26.544      0.000     -12.393     -10.688
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
balance        0.0056      0.000     24.835      0.000       0.005       0.006
==============================================================================

Possibly complete quasi-separation: A fraction 0.14 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

#### (b) Write a function, boot_fn(), that takes as input the Default data set as well as an index of the observations, and that outputs the coefficient estimates for income and balance in the multiple logistic regression model.

In [104]:
def boot_fn(df, idx):

    response = 'default_Yes'
    predictors = ['income', 'balance']

    X = sm.add_constant(np.array(df[predictors].loc[idx]))
    y = np.array(df[response].iloc[idx])

    # Set up the model
    model_logist = sm.Logit(y, X)
    results_logist = model_logist.fit()

    return results_logist.params

#### (c) Following the bootstrap example in the lab, use your boot_fn() function to estimate the standard errors of the logistic regression coefficients for income and balance.


In [132]:
def boot_idx(n):
    """Return index for bootstrap sample of size n
    e.g. generate array in range 0 to n, with replacement"""
    return np.random.randint(low=0, high=n, size=n)

def boot(fn, data_df, samples):
    """Perform bootstrap for B number of samples"""
    results = []
    for s in range(samples):
        Z = fn(data_df, boot_idx(data_df.shape[0]))
        results += [Z]
    return np.array(results)

def standard_deviation(X):
    """Compute deviation error for jth element in matrix X
    equivalent to np.std(X, axis=0)"""
    X_bar = np.mean(X, axis=0)
    SE = np.sqrt((np.sum(np.square(X - X_bar), axis=0)) / (len(X)))
    return SE

B = 10
coef_preds = boot(boot_fn, df_default, samples=B)
coef_pred = np.mean(coef_preds, axis=0)
standard_errs = standard_deviation(coef_preds)

bootstrap_est = pd.DataFrame({'coef_boot': coef_pred, 'SE_boot': standard_errs})
display(bootstrap_est)

Optimization terminated successfully.
         Current function value: 0.074418
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.073702
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078518
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.076127
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.079708
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.074940
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.085552
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078545
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.074123
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.

,coef_boot,SE_boot
0,-11.550890,0.276032
1,0.000018,0.000004
2,0.005699,0.000131


In [129]:
statsmodels_est = pd.DataFrame({'coef_sm': results_logist_6a.params, 'SE_sm': results_logist_6a.bse})
display(statsmodels_est)

,coef_sm,SE_sm
Intercept,-11.540468,0.434772
income,0.000021,0.000005
balance,0.005647,0.000227


### (d) Comment on the estimated standard errors obtained using the glm() function and using your bootstrap function.

The answers are very similar for both methods, even with small bootstrap samples. 

# 7. In Sections 5.1.2 and 5.1.3, we saw that the cross_validate() function can be used in order to compute the LOOCV test error estimate. Alternatively, one could compute those quantities using just sm.GLM() and the predict() method of the fitted model within a for loop. You will now take this approach in order to compute the LOOCV error for a simple logistic regression model on the Weekly data set. Recall that in the context of classiication problems, the LOOCV error is given in (5.4).

### (a) Fit a logistic regression model that predicts Direction using Lag1 and Lag2.

In [135]:
# Import data
df_weekly = pd.read_csv(r'..\Data sets\Weekly.csv')
df_weekly_raw = df_weekly

In [138]:
# Get dummy variables
df_weekly = pd.get_dummies(df_weekly, dtype=np.float64).drop(columns=['Direction_Down'])
df_weekly

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction_Up
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,0.0
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,0.0
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,1.0
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,1.0
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,1.0
...,...,...,...,...,...,...,...,...,...
1084,2010,-0.861,0.043,-2.173,3.599,0.015,3.205160,2.969,1.0
1085,2010,2.969,-0.861,0.043,-2.173,3.599,4.242568,1.281,1.0
1086,2010,1.281,2.969,-0.861,0.043,-2.173,4.835082,0.283,1.0
1087,2010,0.283,1.281,2.969,-0.861,0.043,4.454044,1.034,1.0


In [139]:
# Fit a logistic regression to predict Direction using Lag1 and Lag2
f_7a = 'Direction_Up ~ Lag1 + Lag2'

# Set up the model
model_logist_7a = sm.Logit.from_formula(formula=f_7a, data=df_weekly)
results_logist_7a = model_logist_7a.fit()

display(results_logist_7a.summary())

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           Direction_Up   No. Observations:                 1089
Model:                          Logit   Df Residuals:                     1086
Method:                           MLE   Df Model:                            2
Date:                Sat, 30 Mar 2024   Pseudo R-squ.:                0.005335
Time:                        11:00:08   Log-Likelihood:                -744.11
converged:                       True   LL-Null:                       -748.10
Covariance Type:            nonrobust   LLR p-value:                   0.01848
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2212      0.061      3.599      0.000       0.101       0.342
Lag1          -0.0387      0.026     -1.477      0.140      -0.090       0.013
Lag2           0.0602      0.027      2.270      0.023       0.008       0.112
==============================================================================
"""

#### (b) Fit a logistic regression model that predicts Direction using Lag1 and Lag2 using all but the first observation.

In [241]:
# Create index for LOOCV
train = df_weekly.index > 0

response   = 'Direction_Up'
predictors = ['Lag1', 'Lag2']

# Split into test and train set
X_train = np.array(df_weekly[train][predictors])
X_test  = np.array(df_weekly[~train][predictors])
y_train = np.array(df_weekly[train][response])
y_test  = np.array(df_weekly[~train][response])

# Fit the model
logist_7b       = LogisticRegression(fit_intercept=True)
results_logist_7b = logist_7b.fit(X_train, y_train)

### (c) Use the model from (b) to predict the direction of the first observation. You can do this by predicting that the first observation will go up if P(Direction="Up"|Lag1, Lag2) > 0.5. Was this observation correctly classified?

In [249]:
# Make prediction
y_pred_7b = results_logist_7b.predict(X_test)

In [250]:
#### Confusion matrix
cm_7b = metrics.confusion_matrix(y_test, y_pred_7b)
confusion_matrix(cm_7b)

,Predicted 1,Predicted 0,Total
,,,
Actual 1,0,0,0
Actual 0,1,0,1
Total,1,0,1


The model predicted the direction incorrectly. 

#### (d) Write a for loop from i = 1 to i = n, where n is the number of observations in the data set, that performs each of the following steps:
i. Fit a logistic regression model using all but the ith observation to predict Direction using Lag1 and Lag2.

ii. Compute the posterior probability of the market moving up for the ith observation.

iii. Use the posterior probability for the ith observation in order to predict whether or not the market moves up.

iv. Determine whether or not an error was made in predicting the direction for the ith observation. If an error was made, then indicate this as a 1, and otherwise indicate it as a 0.

#### (e) Take the average of the n numbers obtained in (d)iv in order to obtain the LOOCV estimate for the test error. Comment on the results.